In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import fsolve, least_squares
from contextlib import contextmanager
from io import StringIO

sys.path.append('/path/to/EquiNet') # location of EquiNet repository

from chemprop.train import make_predictions, load_model
from chemprop.args import PredictArgs

In [ ]:
# set your inputs
smiles1 = "CCO"
smiles2 = "O"
P_target = 101325  # Pressure in Pa
n = 101 # Mesh size of mole fractions
model_folder = "." # Directory containing the trained .pt file

In [ ]:
# If you want a temporary path location for the results to write to
import tempfile

temp_preds = tempfile.mktemp(suffix='.csv')
temp_features = tempfile.mktemp(suffix='.csv')
temp_test = tempfile.mktemp(suffix='.csv')

In [ ]:
# context class to suppress print statements from repeated model predictions

class NoPrint:
    """
    Context manager to suppress print statements.
    """
    def __enter__(self):
        self._original_stdout = sys.stdout
        self._original_sterr = sys.stderr
        sys.stdout = StringIO()
        sys.stderr = StringIO()

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self._original_stdout
        sys.stderr = self._original_sterr


In [ ]:
# Finding the pure component vapor pressures over a range of temperatures

logP_target = np.log10(P_target)
print("logP_target", logP_target)

# make input files
x1s = np.full(701, 0.5)
x2s = np.full(701, 0.5)
Ts = np.linspace(100, 800, 701)

test_df = pd.DataFrame({
    'smiles1': [smiles1] * 701,
    'smiles2': [smiles2] * 701,
})
features_df = pd.DataFrame({
    'x1': x1s,
    'x2': x2s,
    'T': Ts,
    'log10P1sat': ["nan"] * 701,
    'log10P2sat': ["nan"] * 701,
})

test_df.to_csv(temp_test, index=False)
features_df.to_csv(temp_features, index=False)

# make predictions

arguments = [
    '--test_path', temp_test,
    '--features_path', temp_features,
    '--preds_path', temp_preds,
    '--checkpoint_dir', model_folder,
    '--number_of_molecules', '2',
    '--num_workers', '0',
]

args = PredictArgs().parse_args(arguments)

model_objects = load_model(args=args)

with NoPrint():
    preds = np.array(make_predictions(args=args, model_objects=model_objects)) # output: [y1,y2,log10P,lngamma1,lngamma2,log10P1sat,log10P2sat]

In [ ]:
# Finding T closest to the target pressure for each component
logP1sats = preds[:, 5]
logP2sats = preds[:, 6]

closest_P1_index = np.argmin(np.abs(logP1sats - logP_target))
closest_P2_index = np.argmin(np.abs(logP2sats - logP_target))

closest_P1_T = Ts[closest_P1_index]
closest_P2_T = Ts[closest_P2_index]

print("closest P1", logP1sats[closest_P1_index], "at T =", closest_P1_T)
print("closest P2", logP2sats[closest_P2_index], "at T =", closest_P2_T)


In [ ]:
# Set up a prediction function to optimize around
counter = 1

def predict_P(T):
    global counter
    global preds

    print(f"iteration {counter}")
    counter += 1
    print("predict_P called with T =", T)
    
    test_df = pd.DataFrame({
        'smiles1': [smiles1]*n,
        'smiles2': [smiles2]*n,
    })
    features_df = pd.DataFrame({
        'x1': np.linspace(0, 1, n),
        'x2': np.linspace(1, 0, n),
        'T': T, # T is a list of temperatures 
        'log10P1sat': ["nan"]*n,
        'log10P2sat': ["nan"]*n,
    })
    test_df.to_csv(temp_test, index=False)
    features_df.to_csv(temp_features, index=False)

    args = PredictArgs().parse_args([
        '--test_path', temp_test,
        '--features_path', temp_features,
        '--preds_path', temp_preds,
        '--checkpoint_dir', model_folder,
        '--number_of_molecules', '2',
        '--num_workers', '0',
    ])

    with NoPrint():
        preds = np.array(make_predictions(args=args, model_objects=model_objects))  # [y1,y2,log10P,lngamma1,lngamma2,log10P1sat,log10P2sat]
        P_preds = 10 ** preds[:, 2]
    print("preds", P_preds)
    return P_preds - P_target

In [ ]:
# Assumes linear interpolation between pure component temperatures
T_guess = np.linspace(closest_P2_T, closest_P1_T, n)
T_guess

In [ ]:
# optimize using scipy. Sparsity indicates that each temperature only affects its own prediction.

T_solution = least_squares(predict_P, T_guess, diff_step=1e-4, jac_sparsity=np.identity(n))

In [ ]:
T = T_solution.x
T

In [ ]:
T_solution.fun # residuals from the target pressure, in Pa

In [ ]:
# Recover compositions from the global preds variable
# To avoid use of global variables, re-predict using the optimized temperatures

x1s = np.linspace(0, 1, n)
y1s = preds[:, 0]


plt.plot(x1s, T,label="liquid", color='blue')
plt.plot(y1s, T, label="vapor", color='red')
plt.legend()
plt.xlim(0, 1)
plt.ylabel('Temperature, T (K)')
plt.xlabel('Mole Fraction of Component 1, x1 and y1')
plt.title(f'Mixture of SMILES {smiles1} (1) and \nSMILES {smiles2} (2) at P = {P_target} Pa')